In [1]:
pip install ultralytics

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install opencv-python numpy filterpy

Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install --upgrade ultralytics opencv-python numpy filterpy

  Using cached numpy-2.2.4-cp312-cp312-win_amd64.whl.metadata (60 kB)
  Using cached numpy-2.1.1-cp312-cp312-win_amd64.whl.metadata (59 kB)
Note: you may need to restart the kernel to use updated packages.


In [50]:
import cv2
import numpy as np
from ultralytics import YOLO
from filterpy.kalman import KalmanFilter

def predict_future_positions(kf, steps=35):
    future_positions = []
    state = kf.x.copy()
    for _ in range(steps):
        state = np.dot(kf.F, state)
        future_positions.append((int(state[0]), int(state[1])))
    return future_positions

model_path = "/Users/Aleksander/Desktop/видео/best.pt"
model = YOLO(model_path)

# Настройка фильтра Калмана
kf = KalmanFilter(dim_x=4, dim_z=2)
kf.F = np.array([[1, 0, 1, 0],
                 [0, 1, 0, 1],
                 [0, 0, 1, 0],
                 [0, 0, 0, 1]])
kf.H = np.array([[1, 0, 0, 0],
                 [0, 1, 0, 0]])
kf.P *= 1000
kf.R = np.array([[10, 0], [0, 10]])
kf.Q *= 0.1

video_path = "/Users/Aleksander/Desktop/видео/IMG_9269.mp4"
output_video_path = "/Users/Aleksander/Desktop/видео/output_video_with_tracking.mp4"

# Открытие видео
cap = cv2.VideoCapture(video_path)

fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fourcc = cv2.VideoWriter_fourcc(*'mp4v')

out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

drone_tracks = []  
max_track_length = 35  

frame_count = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    results = model.predict(source=frame, conf=0.65, iou=0.3)
    annotated_frame = results[0].plot()  # Получаем кадр с bounding boxes от YOLO

    for result in results:
        for box in result.boxes.data:
            x1, y1, x2, y2, conf, cls = box.tolist()
            x_center = (x1 + x2) / 2
            y_center = (y1 + y2) / 2

            # Обновление фильтра Калмана
            kf.predict()
            kf.update([x_center, y_center])
            x_pred, y_pred = kf.x[:2]

            drone_tracks.append((int(x_pred), int(y_pred)))
            if len(drone_tracks) > max_track_length:
                drone_tracks.pop(0)

    # Отрисовка истории перемещения (красная линия)
    for i in range(1, len(drone_tracks)):
        cv2.line(annotated_frame, drone_tracks[i - 1], drone_tracks[i], (0, 0, 255), 2)

    # Предсказание будущих позиций
    future_positions = predict_future_positions(kf, steps=35)

    # Отрисовка предсказанной траектории (синяя линия)
    for i in range(len(future_positions) - 1):
        cv2.line(annotated_frame, future_positions[i], future_positions[i + 1], (255, 0, 0), 2)

    # Сохранение и отображение
    out.write(annotated_frame)
    cv2.imshow("Drone Detection & Tracking", annotated_frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

    frame_count += 1
    print(f"Processed frame {frame_count}")

cap.release()
out.release()
cv2.destroyAllWindows()


0: 640x352 1 drone, 61.6ms
Speed: 1.4ms preprocess, 61.6ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 352)
Processed frame 1

0: 640x352 (no detections), 45.4ms
Speed: 1.2ms preprocess, 45.4ms inference, 0.3ms postprocess per image at shape (1, 3, 640, 352)
Processed frame 2

0: 640x352 (no detections), 46.5ms
Speed: 1.2ms preprocess, 46.5ms inference, 0.3ms postprocess per image at shape (1, 3, 640, 352)
Processed frame 3



C:\Users\Aleksander\AppData\Local\Temp\ipykernel_3964\359153886.py:65: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  drone_tracks.append((int(x_pred), int(y_pred)))
C:\Users\Aleksander\AppData\Local\Temp\ipykernel_3964\359153886.py:11: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  future_positions.append((int(state[0]), int(state[1])))


0: 640x352 (no detections), 44.5ms
Speed: 1.2ms preprocess, 44.5ms inference, 0.3ms postprocess per image at shape (1, 3, 640, 352)
Processed frame 4

0: 640x352 (no detections), 46.2ms
Speed: 1.3ms preprocess, 46.2ms inference, 0.3ms postprocess per image at shape (1, 3, 640, 352)
Processed frame 5

0: 640x352 (no detections), 44.9ms
Speed: 1.2ms preprocess, 44.9ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 352)
Processed frame 6

0: 640x352 (no detections), 45.1ms
Speed: 1.7ms preprocess, 45.1ms inference, 0.3ms postprocess per image at shape (1, 3, 640, 352)
Processed frame 7

0: 640x352 (no detections), 47.1ms
Speed: 1.7ms preprocess, 47.1ms inference, 0.3ms postprocess per image at shape (1, 3, 640, 352)
Processed frame 8

0: 640x352 (no detections), 46.7ms
Speed: 1.2ms preprocess, 46.7ms inference, 0.3ms postprocess per image at shape (1, 3, 640, 352)
Processed frame 9

0: 640x352 (no detections), 44.5ms
Speed: 1.1ms preprocess, 44.5ms inference, 0.3ms postproces